# First Iteration of the Simulation of SAoI

In this notebook, we will perform the first iteration of the SAoI simulation with the aim of determining the optimal signal transmission frequency to minimize SAoI in the M/M/1 queue with a varying Carbon Intensity (CI) index.

In [8]:
import numpy as np 
from numpy import random 
import matplotlib.pyplot as plt 
from math import sin, pi
from collections import deque #We use deque for simulation of a queue

# We need to define function sinus that will represent value of CI over the time. 
def sinus(x):
    """This function simulates value of CI over the time.
    """
    return 100 * sin(x/(2*pi*3600*24)) + 300

def simulacija(ar, tt=36000*24, ts=0.05):
    """Returns the average SAoI (sustainable age of information) among
    time tt with time steps ts for M/M/1 queue with arrival rate ar and 
    serving rate 1.
    
    Parameters
    ----------
    ar: float
        Number between 0 and 1 that represents arrival rate of designed
        M/M/1 queue.
        
    tt: int
        Total time of the simulation.

    ts: float
        Number that represents the time step that we take at each iteration
        of the loop.

    Returns
    --------
    float
        Number that represents the average SAoI.
    """
    #we need list in which all of the times of arrivals are stored. 
    #Times in this list are absolute and not interarrival.
    list_of_arrivals = [0]
    for i in range(2*int(tt/ar)):
        t = list_of_arrivals[-1] + random.exponential(1/ar)
        list_of_arrivals.append(t)

    #we need list in which all of the times of servings are stored.
    #times in this list are interserving times and not absolute times.
    list_of_servings = random.exponential(scale=1, size=2*int(tt))
    ar_ind = 0 #helps us take the element with right index out of list_of_arrivals.
    ser_ind = 0 #helps us take the element with right index out of list_of_servings.
    time = 0 
    sustainable_age_list = [] #list in which we store one value of SAoI for each 100 time units. 
    power = 10 #with which power source works
    age = 0
    age_list = []
    queue = deque([]) 
    inter_serving_time = 0 
    number = 0
    time_elaps = [] #this parameter helps us measure the time remaining till decrease of power function

    #outer loop is the one that will go over whole time. Parameters that are not changing to 0 or empty after 
    #100 time steps should be already defined earlier. 
    for i1 in range(int(tt/100)):
        power_list = [power] #we empty power_list and CI_list at every iteration of an inner loop.
        CI_list = [sinus(time)] #Those two list are to integrate over after 100 time units
        for i2 in range(int(100/ts)):
            #we add a value of one time step (ts) to each of the next parameters because we are moving one 
            #time step ahead in the future.
            time += ts 
            inter_serving_time += ts
            age += ts
            #we evaluate and store value of carbon intensity.
            CI = sinus(time)
            CI_list.append(CI)
            #we decrease each value in the list time_elaps for ts.
            for i in range(len(time_elaps)):
                time_elaps[i] -= ts
            #at this step we check if current time is already bigger than current arrival in the arrival_list.  
            if time > list_of_arrivals[ar_ind]:
                queue.append(time)
                ar_ind += 1
                power += 5
                number += 1
                time_elaps.append(5)
            #at this step we check if inter_serving_time is already bigger than current element and  
            if inter_serving_time > list_of_servings[ser_ind] and len(queue) != 0:
                ser_ind += 1
                inter_serving_time = 0
                age = time - queue[0]
                queue.popleft()
            if time_elaps != []:
                if time_elaps[0] < 0:
                    power -= 5
                    time_elaps = time_elaps[1:]
            power_list.append(power)
            age_list.append(age)
        # we summarize power of last 100 time units
        E = 0
        for e in power_list:
            E += e
        E = E * ts
        # we summarize CI of last 100 time units
        CI_av = 0
        for e in CI_list:
            CI_av += e 
        CI_av = CI_av / len(CI_list)
        CT = CI_av * E 
        # we summarize age of last 100 time units
        ag = 0
        for e in age_list:
            ag += e 
        ag = ag / len(age_list) #ag is average age in last 100 time units
        SAoI = CT * ag #we finnaly evaluate value of SAoI
        sustainable_age_list.append(SAoI)
    return sum(sustainable_age_list) / len(sustainable_age_list)

x_values = []
t = 0.1
for i in range(81):
    x_values.append(t)
    t += 0.01

y_values = []
for e in x_values:
    y_values.append(simulacija(e, tt=36000*24))

x_os = np.array(x_values)
y_os = np.array(y_values)

m = min(y_values)
l = 0
for i in range(81):
    if y_values[i] == m:
        l = 0.1 + i * 0.01

plt.plot(x_os, y_os)
plt.show()
l